## ResNet-18 and CIFAR-10

In [ ]:
'''ResNet in PyTorch.
For Pre-activation ResNet, see 'preact_resnet.py'.
Reference:
[1] Kaiming He, Xiangyu Zhang, Shaoqing Ren, Jian Sun
    Deep Residual Learning for Image Recognition. arXiv:1512.03385
'''
import torch
import torch.nn as nn
import torch.nn.functional as F


class BasicBlock(nn.Module):
    expansion = 1

    def __init__(self, in_planes, planes, stride=1):
        super(BasicBlock, self).__init__()
        self.conv1 = nn.Conv2d(
            in_planes, planes, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(planes)
        self.conv2 = nn.Conv2d(planes, planes, kernel_size=3,
                               stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(planes)

        self.shortcut = nn.Sequential()
        if stride != 1 or in_planes != self.expansion*planes:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_planes, self.expansion*planes,
                          kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(self.expansion*planes)
            )

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.bn2(self.conv2(out))
        out += self.shortcut(x)
        out = F.relu(out)
        return out


class Bottleneck(nn.Module):
    expansion = 4

    def __init__(self, in_planes, planes, stride=1):
        super(Bottleneck, self).__init__()
        self.conv1 = nn.Conv2d(in_planes, planes, kernel_size=1, bias=False)
        self.bn1 = nn.BatchNorm2d(planes)
        self.conv2 = nn.Conv2d(planes, planes, kernel_size=3,
                               stride=stride, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(planes)
        self.conv3 = nn.Conv2d(planes, self.expansion *
                               planes, kernel_size=1, bias=False)
        self.bn3 = nn.BatchNorm2d(self.expansion*planes)

        self.shortcut = nn.Sequential()
        if stride != 1 or in_planes != self.expansion*planes:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_planes, self.expansion*planes,
                          kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(self.expansion*planes)
            )

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = F.relu(self.bn2(self.conv2(out)))
        out = self.bn3(self.conv3(out))
        out += self.shortcut(x)
        out = F.relu(out)
        return out


class ResNet(nn.Module):
    def __init__(self, block, num_blocks, num_classes=10):
        super(ResNet, self).__init__()
        self.in_planes = 64

        self.conv1 = nn.Conv2d(3, 64, kernel_size=3,
                               stride=1, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(64)
        self.layer1 = self._make_layer(block, 64, num_blocks[0], stride=1)
        self.layer2 = self._make_layer(block, 128, num_blocks[1], stride=2)
        self.layer3 = self._make_layer(block, 256, num_blocks[2], stride=2)
        self.layer4 = self._make_layer(block, 512, num_blocks[3], stride=2)
        self.linear = nn.Linear(512*block.expansion, num_classes)

    def _make_layer(self, block, planes, num_blocks, stride):
        strides = [stride] + [1]*(num_blocks-1)
        layers = []
        for stride in strides:
            layers.append(block(self.in_planes, planes, stride))
            self.in_planes = planes * block.expansion
        return nn.Sequential(*layers)

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.layer4(out)
        out = F.avg_pool2d(out, 4)
        out = out.view(out.size(0), -1)
        out = self.linear(out)
        return out


def ResNet18():
    return ResNet(BasicBlock, [2, 2, 2, 2])


def ResNet34():
    return ResNet(BasicBlock, [3, 4, 6, 3])


def ResNet50():
    return ResNet(Bottleneck, [3, 4, 6, 3])


def ResNet101():
    return ResNet(Bottleneck, [3, 4, 23, 3])


def ResNet152():
    return ResNet(Bottleneck, [3, 8, 36, 3])

In [ ]:
'''Some helper functions for PyTorch, including:
    - get_mean_and_std: calculate the mean and std value of dataset.
    - msr_init: net parameter initialization.
    - progress_bar: progress bar mimic xlua.progress.
'''
import os
import sys
import time
import math

import torch.nn as nn
import torch.nn.init as init


def get_mean_and_std(dataset):
    '''Compute the mean and std value of dataset.'''
    dataloader = torch.utils.data.DataLoader(dataset, batch_size=1, shuffle=True, num_workers=2)
    mean = torch.zeros(3)
    std = torch.zeros(3)
    print('==> Computing mean and std..')
    for inputs, targets in dataloader:
        for i in range(3):
            mean[i] += inputs[:,i,:,:].mean()
            std[i] += inputs[:,i,:,:].std()
    mean.div_(len(dataset))
    std.div_(len(dataset))
    return mean, std

def init_params(net):
    '''Init layer parameters.'''
    for m in net.modules():
        if isinstance(m, nn.Conv2d):
            init.kaiming_normal(m.weight, mode='fan_out')
            if m.bias:
                init.constant(m.bias, 0)
        elif isinstance(m, nn.BatchNorm2d):
            init.constant(m.weight, 1)
            init.constant(m.bias, 0)
        elif isinstance(m, nn.Linear):
            init.normal(m.weight, std=1e-3)
            if m.bias:
                init.constant(m.bias, 0)

def format_time(seconds):
    days = int(seconds / 3600/24)
    seconds = seconds - days*3600*24
    hours = int(seconds / 3600)
    seconds = seconds - hours*3600
    minutes = int(seconds / 60)
    seconds = seconds - minutes*60
    secondsf = int(seconds)
    seconds = seconds - secondsf
    millis = int(seconds*1000)

    f = ''
    i = 1
    if days > 0:
        f += str(days) + 'D'
        i += 1
    if hours > 0 and i <= 2:
        f += str(hours) + 'h'
        i += 1
    if minutes > 0 and i <= 2:
        f += str(minutes) + 'm'
        i += 1
    if secondsf > 0 and i <= 2:
        f += str(secondsf) + 's'
        i += 1
    if millis > 0 and i <= 2:
        f += str(millis) + 'ms'
        i += 1
    if f == '':
        f = '0ms'
    return f

In [ ]:
'''Train CIFAR10 with PyTorch.'''
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torch.backends.cudnn as cudnn

import torchvision
import torchvision.transforms as transforms

import os
import argparse
import csv

device = 'cuda' if torch.cuda.is_available() else 'cpu'
best_acc = 0  # best test accuracy
start_epoch = 0  # start from epoch 0 or last checkpoint epoch

# Data
print('==> Preparing data..')
transform_train = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

trainset = torchvision.datasets.CIFAR10(
    root='./data', train=True, download=True, transform=transform_train)
trainloader = torch.utils.data.DataLoader(
    trainset, batch_size=128, shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(
    root='./data', train=False, download=True, transform=transform_test)
testloader = torch.utils.data.DataLoader(
    testset, batch_size=1, shuffle=False, num_workers=2)

classes = ('plane', 'car', 'bird', 'cat', 'deer',
           'dog', 'frog', 'horse', 'ship', 'truck')

# Model
print('==> Building model..')
# net = VGG('VGG19')
net = ResNet18()
# net = PreActResNet18()
# net = GoogLeNet()
# net = DenseNet121()
# net = ResNeXt29_2x64d()
# net = MobileNet()
#net = MobileNetV2()
# net = DPN92()
# net = ShuffleNetG2()
# net = SENet18()
# net = ShuffleNetV2(1)
# net = EfficientNetB0()
# net = RegNetX_200MF()
net = net.to(device)
if device == 'cuda':
    net = torch.nn.DataParallel(net)
    cudnn.benchmark = True

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001,
                      momentum=0.9, weight_decay=5e-4)
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=200)


# Training
def train(epoch):
    print('\nEpoch: %d' % epoch)
    net.train()
    train_loss = 0
    correct = 0
    total = 0
    for batch_idx, (inputs, targets) in enumerate(trainloader):
        inputs, targets = inputs.to(device), targets.to(device)
        optimizer.zero_grad()
        outputs = net(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()

        train_loss += loss.item()
        _, predicted = outputs.max(1)
        total += targets.size(0)
        correct += predicted.eq(targets).sum().item()
        if batch_idx % 50 == 0:
          with open('log_baseline_train.csv', 'a') as f:
                writer = csv.writer(f)
                writer.writerow([batch_idx + epoch * 400, train_loss/(batch_idx+1), correct/total])
          print("Epoch", epoch, 'iteration',batch_idx, 'Loss: %.3f | Acc: %.3f%% (%d/%d)'
                        % (train_loss/(batch_idx+1), 100.*correct/total, correct, total))


def test(epoch):
    global best_acc
    net.eval()
    test_loss = 0
    correct = 0
    total = 0
    with torch.no_grad():
        for batch_idx, (inputs, targets) in enumerate(testloader):
            inputs, targets = inputs.to(device), targets.to(device)
            outputs = net(inputs)
            loss = criterion(outputs, targets)

            test_loss += loss.item()
            _, predicted = outputs.max(1)
            total += targets.size(0)
            correct += predicted.eq(targets).sum().item()

            if batch_idx % 1000 == 0:
              with open('log_baseline_test.csv', 'a') as f:
                writer = csv.writer(f)
                writer.writerow([batch_idx + epoch * 100, test_loss/(batch_idx+1), correct/total])
              print("Epoch", epoch, 'iteration',batch_idx, 'Loss: %.3f | Acc: %.3f%% (%d/%d)'
                          % (test_loss/(batch_idx+1), 100.*correct/total, correct, total))

    # Save checkpoint.
    acc = 100.*correct/total
    if acc > best_acc:
        print('Saving..')
        state = {
            'net': net.state_dict(),
            'acc': acc,
            'epoch': epoch,
        }
        if not os.path.isdir('checkpoint'):
            os.mkdir('checkpoint')
        torch.save(state, './checkpoint/ckpt.pth')
        best_acc = acc

with open('log_baseline_train.csv', 'w') as f:
        writer = csv.writer(f)
        writer.writerow(["iteration", "train_loss", "train_acc"])

with open('log_baseline_test.csv', 'w') as f:
        writer = csv.writer(f)
        writer.writerow(["iteration", "test_loss", "test_acc"])

for epoch in range(start_epoch, start_epoch+35):
    train(epoch)
    test(epoch)
    scheduler.step()

==> Preparing data..
Files already downloaded and verified
Files already downloaded and verified
==> Building model..

Epoch: 0
Epoch 0 iteration 0 Loss: 2.339 | Acc: 15.625% (20/128)
Epoch 0 iteration 50 Loss: 2.111 | Acc: 21.661% (1414/6528)
Epoch 0 iteration 100 Loss: 1.956 | Acc: 27.599% (3568/12928)
Epoch 0 iteration 150 Loss: 1.852 | Acc: 31.188% (6028/19328)
Epoch 0 iteration 200 Loss: 1.780 | Acc: 33.745% (8682/25728)
Epoch 0 iteration 250 Loss: 1.725 | Acc: 35.825% (11510/32128)
Epoch 0 iteration 300 Loss: 1.680 | Acc: 37.689% (14521/38528)
Epoch 0 iteration 350 Loss: 1.637 | Acc: 39.352% (17680/44928)
Epoch 0 iteration 0 Loss: 0.752 | Acc: 100.000% (1/1)
Epoch 0 iteration 1000 Loss: 1.343 | Acc: 51.948% (520/1001)
Epoch 0 iteration 2000 Loss: 1.365 | Acc: 49.775% (996/2001)
Epoch 0 iteration 3000 Loss: 1.365 | Acc: 49.450% (1484/3001)
Epoch 0 iteration 4000 Loss: 1.362 | Acc: 49.463% (1979/4001)
Epoch 0 iteration 5000 Loss: 1.357 | Acc: 50.190% (2510/5001)
Epoch 0 iteration 6

KeyboardInterrupt: ignored

In [ ]:
torch.save(net.state_dict(), "/content/model5.pth")

## Crossbar

In [ ]:
"""
crossbar.py
Louis Primeau
University of Toronto Department of Electrical and Computer Engineering
louis.primeau@mail.utoronto.ca
July 29th 2020
"""
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import itertools
import time
import seaborn as sns
import matplotlib.pyplot as plt
from scipy.io import savemat
import torchvision
import torch.optim as optim
import math
from scipy.io import savemat

# Implements scipy's minmax scaler except just between 0 and 1 for torch Tensors.
# Taken from a ptrblck post on the PyTorch forums. Love that dude.
class MinMaxScaler(object):
    def __call__(self, tensor):
        self.scale = 1.0 / (tensor.max(dim=1, keepdim=True)[0] - tensor.min(dim=1, keepdim=True)[0])
        self.min = tensor.min(dim=1, keepdim=True)[0]
        tensor.sub_(self.min).mul_(self.scale)
        return tensor
    def inverse_transform(self, tensor):
        tensor.div_(self.scale).add_(self.min)
        return tensor


In [ ]:
class ticket:
    def __init__(self, row, col, m_rows, m_cols, matrix, mat_scale_factor, crossbar, uvect, decode, inputres, outputres):
        self.row, self.col = row, col
        self.m_rows, self.m_cols = m_rows, m_cols
        self.crossbar = crossbar
        self.mat_scale_factor = mat_scale_factor
        self.matrix = matrix
        self.uvect = uvect
        self.inputres = inputres
        self.adcres = outputres
        self.decode = torch.matmul(self.uvect.t(),self.matrix)

    def prep_vector(self, vector, v_bits):

        # Scale vector to [0, 2^v_bits]
        vect_min = torch.min(vector)
        vector = vector - vect_min        
        vect_scale_factor = torch.max(vector) / (2**v_bits - 1)
        vector = vector / vect_scale_factor if vect_scale_factor != 0.0 else vector

        # decompose vector by bit
        bit_vector = torch.zeros(vector.size(0),v_bits)
        bin2s = lambda x : ''.join(reversed( [str((int(x) >> i) & 1) for i in range(v_bits)] ) )
        for j in range(vector.size(0)):
            bit_vector[j,:] = torch.Tensor([float(i) for i in list(bin2s(vector[j]))])
        bit_vector *= self.crossbar.V
        
        # Pad bit vector with unselected voltages
        pad_vector = torch.zeros(self.crossbar.size[0], v_bits)
        
        pad_vector[self.row:self.row + self.m_rows,:] = bit_vector

        return pad_vector, vect_scale_factor, vect_min

    def vmm(self, vector):
        # Baseline VMM operation without CODEX
        v_bits = self.inputres
        assert vector.size(1) == 1, "vector wrong shape"

        crossbar = self.crossbar
        # Rescale vector and convert to bits.
        pad_vector, vect_scale_factor, vect_min = self.prep_vector(vector, v_bits)

        rW = self.crossbar.W[0:(self.matrix.shape[0]),0:(2*self.matrix.shape[1])]
        rW = rW[:,1::2] - rW[:,0::2]

        # Perform crossbar VMM
        rV = torch.transpose(pad_vector[0:vector.size(0)],0,1)
        rout = torch.matmul(rV, rW)

        # Round rout to input ADC resolution     
        rout_scale_factor = torch.max(rout) / (2**self.adcres - 1)
        rout = rout / rout_scale_factor
        rout = torch.round(rout)
        rout = rout * rout_scale_factor

        # Add binary outputs
        for i in range(rout.size(0)):
            rout[i] *= 2**(v_bits - i - 1)
        rout = torch.sum(rout, axis=0)

        # Rescale binary outputs
        rout = (rout / crossbar.V * vect_scale_factor*self.mat_scale_factor) / 1.5131 + torch.sum(vect_min*self.matrix,axis=0)
        return rout.view(-1,1)
    
    def CODEXvmm(self, xvector):
        # CODEX VMM operation
        assert xvector.size(1) == 1, "vector wrong shape"
        v_bits=self.inputres
        crossbar = self.crossbar

        #Add encoding vector u to x
        vector = xvector + self.uvect
        pad_vector, vect_scale_factor, vect_min = self.prep_vector(vector, v_bits+1)

        rW = self.crossbar.W[0:(self.matrix.shape[0]),0:(2*self.matrix.shape[1])]
        rW = rW[:,1::2] - rW[:,0::2]

        rV = torch.transpose(pad_vector[0:vector.size(0)],0,1)
        # The rout on the line below this comment contains 
        # the raw output currents that the ADC will receive.
        rout = torch.matmul(rV, rW)

        # Round rout to input ADC resolution     
        rout_scale_factor = torch.max(rout) / (2**self.adcres - 1)
        rout = rout / rout_scale_factor
        rout = torch.round(rout)
        rout = rout * rout_scale_factor

        for i in range(rout.size(0)):
            rout[i] *= 2**(v_bits - i - 1)
        rout = torch.sum(rout, axis=0)
        rout = 2*(rout / crossbar.V * vect_scale_factor*self.mat_scale_factor) / 1.5231 + torch.sum(vect_min*self.matrix,axis=0)
        rout = rout - self.decode
        return rout.view(-1,1)     

    def modified_CODEXvmm(self, xvector):
        # CODEX VMM operation
        assert xvector.size(1) == 1, "vector wrong shape"
        v_bits=self.inputres
        crossbar = self.crossbar

        #Add encoding vector u to x
        vector = xvector + self.uvect
        pad_vector, vect_scale_factor, vect_min = self.prep_vector(vector, v_bits+1)

        rW = self.crossbar.W[0:(self.matrix.shape[0]),0:(2*self.matrix.shape[1])]
        rW = rW[:,1::2] - rW[:,0::2]

        rV = torch.transpose(pad_vector[0:vector.size(0)],0,1)
        # The rout on the line below this comment contains 
        # the raw output currents that the ADC will receive.
        rout = torch.matmul(rV, rW)

        # Round rout to input ADC resolution     
        rout_scale_factor = torch.max(rout) / (2**self.adcres - 1)
        rout = rout / rout_scale_factor
        rout = torch.round(rout)
        rout = rout * rout_scale_factor

        for i in range(rout.size(0)):
            rout[i] *= 2**(v_bits - i - 1)
        rout = torch.sum(rout, axis=0)
        rout = 2*(rout / crossbar.V * vect_scale_factor*self.mat_scale_factor) / 1.5231 + torch.sum(vect_min*self.matrix,axis=0)
        
        # do not decode except during inference
        # rout = rout - self.decode 
        return rout.view(-1,1)     

In [ ]:
import torch.optim as optim
import random
import copy

class linear(torch.autograd.Function):
    #From Louis: Custom pytorch autograd function for crossbar VMM operation
    @staticmethod
    def forward(ctx, ticket, x, W, b):
        ctx.save_for_backward(x, W, b)
        return ticket.CODEXvmm(x) + b
        
    @staticmethod
    def backward(ctx, dx):
        x, W, b = ctx.saved_tensors
        grad_input = W.t().mm(dx)
        grad_weight = dx.mm(x.t())
        grad_bias = dx
        return (None, grad_input, grad_weight, grad_bias)

class Linear(torch.nn.Module):
    def __init__(self, input_size, output_size, cb,uvect):
        super(Linear, self).__init__()
        self.W = torch.nn.parameter.Parameter(torch.rand(output_size, input_size))
        self.b = torch.nn.parameter.Parameter(torch.rand(output_size, 1))
        self.cb = cb

        #Instantiate Linear layer with pool of random encoding vectors to sample from
        self.uvectlist = uvect
        self.uvectidx = 0
        # Decoding vector is calculated ideally here off-chip, but calculating decoding vector on-chip is also possible
        self.decode = torch.matmul(self.uvectlist[self.uvectidx].t(),torch.transpose(self.W,0,1)).detach().clone()
        self.ticket = cb.register_linear(torch.transpose(self.W,0,1),self.uvectlist[self.uvectidx],self.decode)
        self.f = linear()
        self.cbon = False
        
    def forward(self, x):
        return self.f.apply(self.ticket, x, self.W, self.b) if self.cbon else self.W.matmul(x) + self.b

    def remap(self):
        #Should call the remap crossbar function after 1 or a couple update steps 
        self.cb.clear()
        self.ticket = self.cb.register_linear(torch.transpose(self.W,0,1),self.uvectlist[self.uvectidx],self.decode)

    def update_decode(self):
        #Update decoding vector by updating U*G. 
        self.decode = torch.matmul(self.uvectlist[self.uvectidx].t(),torch.transpose(self.W,0,1)).detach().clone()

    def resample(self):
        #Sample random new uvector from provided uvectlist
        self.cb.clear()
        self.uvectidx = random.randint(0, len(uvectlist)-1)
        self.ticket = self.cb.register_linear(torch.transpose(self.W,0,1),self.uvectlist[self.uvectidx],self.decode)
    
    def use_cb(self, state):
        self.cbon = state


In [ ]:
# Key Idea is that CODEX allows us to use higher ADC inpt resolution by
# Reducing the ADC sensing range.
device_params = {"Vdd": 1.8,
                 "r_wl": 20,
                 "r_bl": 20,
                 "m": 600,
                 "n": 600,
                 "r_on_mean": 1e4,
                 "r_on_stddev": 1e3,
                 "r_off_mean": 1e5,
                 "r_off_stddev": 1e4,
                 "dac_resolution": 5,
                 "adc_resolution": 8.3,
                 "device_resolution": 6,
                 "bias_scheme": 1/3,
                 "tile_rows": 4,
                 "tile_cols": 4,
                 "r_cmos_line": 600,
                 "r_cmos_transistor": 20, 
                 "p_stuck_on": 0.01,
                 "p_stuck_off": 0.01}


class crossbar:
    def __init__(self, device_params):
  
        # Power Supply Voltage
        self.V = device_params["Vdd"]

        # DAC resolution
        self.input_resolution = device_params["dac_resolution"]
        self.output_resolution = device_params["adc_resolution"]

        # Wordline Resistance 
        self.r_wl = torch.Tensor((device_params["r_wl"],))
        # Bitline Resistance
        self.r_bl = torch.Tensor((device_params["r_bl"],))

        # Number of rows, columns
        self.size = device_params["m"], device_params["n"]

        # High resistance state
        self.g_on = 1 / torch.normal(device_params["r_on_mean"], device_params["r_on_stddev"], size=self.size)
        #self.g_on = (1 / device_params["r_on_mean"]) * torch.ones(self.size)
        
        # Low Resistance state
        self.g_off = 1 / torch.normal(device_params["r_off_mean"], device_params["r_off_stddev"], size=self.size)
        #self.g_off = (1 / device_params["r_off_mean"]) * torch.ones(self.size)
        
        self.g_wl = torch.Tensor((1 / device_params["r_wl"],))
        self.g_bl = torch.Tensor((1 / device_params["r_bl"],))
        
        # Resolution
        self.resolution = device_params["device_resolution"]
        # Conductance tensor, m x n x 2**resolution        

        # 2**self.resolution - 1 so that there's a conductance state in the middle.
        self.conductance_states = torch.cat([torch.cat([torch.linspace(self.g_off[i,j], self.g_on[i,j],2**self.resolution - 1).unsqueeze(0)
                                                        for j in range(self.size[1])],dim=0).unsqueeze(0)
                                             for i in range(self.size[0])],dim=0)

        # Bias Scheme
        self.bias_voltage = self.V * device_params["bias_scheme"]
        
        # Tile size (1x1 = 1T1R, nxm = passive, etc.)
        self.tile_rows = device_params["tile_rows"]
        self.tile_cols = device_params["tile_cols"]
        assert self.size[0] % self.tile_rows == 0, "tile size does not divide crossbar size in row direction"
        assert self.size[1] % self.tile_cols == 0, "tile size does not divide crossbar size in col direction"
        
        # Resistance of CMOS lines
        self.r_cmos_line = device_params["r_cmos_line"]

        # Conductance Matrix; initialize each memristor at the on resstance
        self.W = torch.ones(self.size) * self.g_on

        # Stuck-on & stuck-on device nonideality 
        self.p_stuck_on = device_params["p_stuck_on"]
        self.p_stuck_off = device_params["p_stuck_off"]
        self.devicefaults = False

        self.mapped = []
        
        self.saved_tiles = {}
        
    def apply_stuck(self, p_stuck_on, p_stuck_off):

        state_dist = torch.distributions.categorical.Categorical(probs=torch.Tensor([p_stuck_on, p_stuck_off, 1 - p_stuck_on - p_stuck_off]))
        state_mask = state_dist.sample(self.size)

        self.W[state_mask == 0] = self.g_off[state_mask==0]
        self.W[state_mask == 1] = self.g_on[state_mask==1]
        
        return None
    
    def map(self, matrix):
        assert not(matrix.size(0) > self.size[0] or matrix.size(1)*2 > self.size[1]), "input too large"
        midpoint = self.conductance_states.size(2) // 2
        
        for i in range(matrix.size(0)):
            for j in range(matrix.size(1)):

                shifted = self.conductance_states[i,j] - self.conductance_states[i,j,midpoint]
                idx = torch.min(torch.abs(shifted - matrix[i,j]), dim=0)[1]    

                self.W[i,2*j+1] = self.conductance_states[i,j,idx]
                self.W[i,2*j] = self.conductance_states[i,j,midpoint-(idx-midpoint)]

    def solve(self, voltage):
        output = torch.zeros((voltage.size(1), self.size[1]))
        for i in range(self.size[0] // self.tile_rows):
            for j in range(self.size[1] // self.tile_cols):
                for k in range(voltage.size(1)):
                    coords = (i*self.tile_rows, (i+1)*self.tile_rows, j*self.tile_cols, (j+1)*self.tile_rows)
                    vect = voltage[i*self.tile_rows:(i+1)*self.tile_rows,k]
                    solution = self.circuit_solve(coords, vect, torch.zeros(self.size[1]), torch.ones(self.size[1]), torch.zeros(self.size[0]))
                    output[k] += torch.cat((torch.zeros(j*self.tile_cols), solution, torch.zeros((self.size[1] // self.tile_cols - j - 1) * self.tile_cols)))
        return output

    """
    A Comprehensive Crossbar Array Model With Solutions for Line Resistance and Nonlinear Device Characteristics
    An Chen
    IEEE TRANSACTIONS ON ELECTRON DEVICES, VOL. 60, NO. 4, APRIL 2013
    """
    
    def hash_M(self, a, b, c, d):
        return str(a) + "_" + str(b) + "_" + str(c) + "_" + str(d)
    
    def make_M(self, a, b, c, d):
        
        conductances = self.W[a:b,c:d]
        g_wl, g_bl = self.g_wl, self.g_bl
        g_s_wl_in, g_s_wl_out = torch.ones(self.tile_rows) * 1, torch.ones(self.tile_rows) * 1e-9
        g_s_bl_in, g_s_bl_out = torch.ones(self.tile_rows) * 1e-9, torch.ones(self.tile_rows) * 1
        m, n = self.tile_rows, self.tile_cols
        
        A = torch.block_diag(*tuple(torch.diag(conductances[i,:])
                          + torch.diag(torch.cat((g_wl, g_wl * 2 * torch.ones(n-2), g_wl)))
                          + torch.diag(g_wl * -1 *torch.ones(n-1), diagonal = 1)
                          + torch.diag(g_wl * -1 *torch.ones(n-1), diagonal = -1)
                          + torch.diag(torch.cat((g_s_wl_in[i].view(1), torch.zeros(n - 2), g_s_wl_out[i].view(1))))
                                   for i in range(m)))

        B = torch.block_diag(*tuple(-torch.diag(conductances[i,:]) for i in range(m)))
        
        def makec(j):
            c = torch.zeros(m, m*n)
            for i in range(m):
                c[i,n*(i) + j] = conductances[i,j]
            return c
  
        C = torch.cat([makec(j) for j in range(n)],dim=0)
        
        def maked(j):
            d = torch.zeros(m, m*n)

            def c(k): 
                return(k - 1)
            
            i = 1
            d[c(i),c(j)] = -g_s_bl_in[c(j)] - g_bl - conductances[c(i),c(j)]
            d[c(i), n*i + c(j)] = g_bl

            i = m
            d[c(i), n*(i-2) + c(j)] = g_bl
            d[c(i), n*(i-1) + c(j)] = -g_s_bl_out[c(j)] - conductances[c(i),c(j)] - g_bl

            for i in range(2, m):
                d[c(i), n*(i-2) + c(j)] = g_bl
                d[c(i), n*(i-1) + c(j)] = -g_bl - conductances[c(i),c(j)] - g_bl
                d[c(i), n*(i+1) + c(j)] = g_bl

            return d

        D = torch.cat([maked(j) for j in range(1,n+1)], dim=0)

        M = torch.cat((torch.cat((A,B),dim=1), torch.cat((C,D),dim=1)), dim=0)
        
        self.saved_tiles[self.hash_M(a,b,c,d)] = M
        
        return torch.inverse(M)
    
    def circuit_solve(self, coords,  v_wl_in, v_bl_in, v_bl_out, v_wl_out):
        
        g_wl, g_bl = self.g_wl, self.g_bl
        g_s_wl_in, g_s_wl_out = torch.ones(self.tile_rows) * 1, torch.ones(self.tile_rows) * 1e-9
        g_s_bl_in, g_s_bl_out = torch.ones(self.tile_rows) * 1e-9, torch.ones(self.tile_rows) * 1
        m, n = self.tile_rows, self.tile_cols
        
        
        if self.hash_M(*coords) not in self.saved_tiles.keys():
            #print(coords)
            M = self.make_M(*coords)
        else:
            M = self.saved_tiles[self.hash_M(*coords)]
        
        E = torch.cat([torch.cat(((v_wl_in[i]*g_s_wl_in[i]).view(1), #EW
                                  torch.zeros(n-2),
                                  (v_wl_out[i]*g_s_wl_out[i]).view(1)))
                                 for i in range(m)] +
                      [torch.cat(((-v_bl_in[i]*g_s_bl_in[i]).view(1), #EB
                                  torch.zeros(m-2),
                                  (-v_bl_in[i]*g_s_bl_out[i]).view(1)))
                                 for i in range(n)]
        ).view(-1, 1)
        
        V = torch.matmul(M, E)
        
        V = torch.chunk(torch.solve(E, M)[0], 2)

        return torch.sum((V[1] - V[0]).view(m,n)*self.W[coords[0]:coords[1],coords[2]:coords[3]],dim=0)

    def register_linear(self, matrix, uvectlist, decode, bias=None):

        row, col = self.find_space(matrix.size(0), matrix.size(1))
        # Need to add checks for bias size and col size
        
        # Scale matrix                                    
        mat_scale_factor = torch.max(torch.abs(matrix)) / torch.max(self.g_on) * 2
        scaled_matrix = matrix / mat_scale_factor
        
        midpoint = self.conductance_states.size(2) // 2
        for i in range(row, row + scaled_matrix.size(0)):
            for j in range(col, col + scaled_matrix.size(1)):
                
                shifted = self.conductance_states[i,j] - self.conductance_states[i,j,midpoint]
                idx = torch.min(torch.abs(shifted - scaled_matrix[i-row,j-col]), dim=0)[1]
                self.W[i,2*j+1] = self.conductance_states[i,j,idx]
                self.W[i,2*j] = self.conductance_states[i,j,midpoint-(idx-midpoint)]
        
        return ticket(row, col, matrix.size(0), matrix.size(1), matrix, mat_scale_factor, self, uvectlist, decode, self.input_resolution, self.output_resolution)

    def which_tiles(self, row, col, m_row, m_col):
        return itertools.product(range(row // self.tile_rows, (row + m_row) // self.tile_rows + 1),
                                 range(col // self.tile_cols,(col + m_col) // self.tile_cols + 1),
        )

    def find_space(self, m_row, m_col):
        if not self.mapped:
            self.mapped.append((0,0,m_row,m_col))
        else:
            self.mapped.append((self.mapped[-1][0] + self.mapped[-1][2], self.mapped[-1][1] + self.mapped[-1][3], m_row, m_col))
        return self.mapped[-1][0], self.mapped[-1][1] 
    
    def clear(self):
        self.mapped = []
        self.W = torch.ones(self.size) * self.g_on

    def conductance_update(self):
        self.conductance_states = torch.cat([torch.cat([torch.linspace(self.g_off[i,j], self.g_on[i,j],2**self.resolution - 1).unsqueeze(0)
                                                        for j in range(self.size[1])],dim=0).unsqueeze(0)
                                             for i in range(self.size[0])],dim=0)

In [ ]:
'''ResNet in PyTorch.
For Pre-activation ResNet, see 'preact_resnet.py'.
Reference:
[1] Kaiming He, Xiangyu Zhang, Shaoqing Ren, Jian Sun
    Deep Residual Learning for Image Recognition. arXiv:1512.03385
'''
import torch
import torch.nn as nn
import torch.nn.functional as F


class BasicBlock(nn.Module):
    expansion = 1

    def __init__(self, in_planes, planes, stride=1):
        super(BasicBlock, self).__init__()
        self.conv1 = nn.Conv2d(
            in_planes, planes, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(planes)
        self.conv2 = nn.Conv2d(planes, planes, kernel_size=3,
                               stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(planes)

        self.shortcut = nn.Sequential()
        if stride != 1 or in_planes != self.expansion*planes:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_planes, self.expansion*planes,
                          kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(self.expansion*planes)
            )

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.bn2(self.conv2(out))
        out += self.shortcut(x)
        out = F.relu(out)
        return out


class Bottleneck(nn.Module):
    expansion = 4

    def __init__(self, in_planes, planes, stride=1):
        super(Bottleneck, self).__init__()
        self.conv1 = nn.Conv2d(in_planes, planes, kernel_size=1, bias=False)
        self.bn1 = nn.BatchNorm2d(planes)
        self.conv2 = nn.Conv2d(planes, planes, kernel_size=3,
                               stride=stride, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(planes)
        self.conv3 = nn.Conv2d(planes, self.expansion *
                               planes, kernel_size=1, bias=False)
        self.bn3 = nn.BatchNorm2d(self.expansion*planes)

        self.shortcut = nn.Sequential()
        if stride != 1 or in_planes != self.expansion*planes:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_planes, self.expansion*planes,
                          kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(self.expansion*planes)
            )

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = F.relu(self.bn2(self.conv2(out)))
        out = self.bn3(self.conv3(out))
        out += self.shortcut(x)
        out = F.relu(out)
        return out


class ResNetM(nn.Module):
    def __init__(self, block, num_blocks, num_classes=10):
        super(ResNetM, self).__init__()
        self.in_planes = 64

        self.conv1 = nn.Conv2d(1, 64, kernel_size=3,
                               stride=1, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(64)
        self.layer1 = self._make_layer(block, 64, num_blocks[0], stride=1)
        self.layer2 = self._make_layer(block, 128, num_blocks[1], stride=2)
        self.layer3 = self._make_layer(block, 256, num_blocks[2], stride=2)
        self.layer4 = self._make_layer(block, 512, num_blocks[3], stride=2)
        # self.linear = nn.Linear(512*block.expansion, num_classes)
        self.uvect = [2*(torch.rand(512,1) - 0.5) for i in range(0,10)] # 
        crb1 = crossbar(device_params)
        # Can test using more than 1 crossbar linear layers.
        # Easiest implementation is to create a crossbar for each linear layer
        self.fc1 = Linear(512, 10,crb1,self.uvect)
        self.fc1.use_cb(True)
        #self.fc2 = nn.Linear(64*2*2, 10)
        self.traincount = 0

    def _make_layer(self, block, planes, num_blocks, stride):
        strides = [stride] + [1]*(num_blocks-1)
        layers = []
        for stride in strides:
            layers.append(block(self.in_planes, planes, stride))
            self.in_planes = planes * block.expansion
        return nn.Sequential(*layers)

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.layer4(out)
        out = F.avg_pool2d(out, 4)
        out = out.view(512, 1)
        #print(out.shape)
        out = self.fc1(out)
        out = out.t()
        
        out = F.log_softmax(out)
        return out


def ResNet18M():
    return ResNetM(BasicBlock, [2, 2, 2, 2])


def ResNet34M():
    return ResNetM(BasicBlock, [3, 4, 6, 3])


def ResNet50M():
    return ResNetM(Bottleneck, [3, 4, 6, 3])


def ResNet101M():
    return ResNetM(Bottleneck, [3, 4, 23, 3])


def ResNet152M():
    return ResNetM(Bottleneck, [3, 8, 36, 3])

In [ ]:
state_dict = torch.load('model_mnist.pth')
temp_model = ResNet18()
#Net2.load_state_dict(torch.load("model5.pth"))

from collections import OrderedDict
new_state_dict = OrderedDict()
for k, v in state_dict.items():
    name = k[7:] # remove `module.`
    new_state_dict[name] = v
# load params
temp_model.load_state_dict(new_state_dict)

<All keys matched successfully>

In [ ]:
state_dict = torch.load('model_mnist.pth')
model = ResNet18M()
# Net2.load_state_dict(torch.load("model5.pth"))

from collections import OrderedDict
new_state_dict = OrderedDict()
for k, v in state_dict.items():
    name = k[7:] # remove `module.`
    new_state_dict[name] = v
# load params
model.load_state_dict(new_state_dict)

RuntimeError: ignored

In [ ]:
# crossbar update
device_params = {"Vdd": 1.8,
                 "r_wl": 20,
                 "r_bl": 20,
                 "m": 600,
                 "n": 600,
                 "r_on_mean": 1e4,
                 "r_on_stddev": 1e3,
                 "r_off_mean": 1e5,
                 "r_off_stddev": 1e4,
                 "dac_resolution": 5,
                 "adc_resolution": 8.3,
                 "device_resolution": 6,
                 "bias_scheme": 1/3,
                 "tile_rows": 4,
                 "tile_cols": 4,
                 "r_cmos_line": 600,
                 "r_cmos_transistor": 20, 
                 "p_stuck_on": 0.01,
                 "p_stuck_off": 0.01}
crb_new = crossbar(device_params)

model.fc1.W = torch.nn.parameter.Parameter(temp_model.linear.weight.data)
model.fc1.b = torch.nn.parameter.Parameter(temp_model.linear.bias.data)

model.fc1.cb = crb_new
model.fc1.remap()

In [ ]:
model.eval()

ResNetM(
  (conv1): Conv2d(1, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (shortcut): Sequential()
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine

In [ ]:
def network_tester(model, test_loader, test_size, epoch, log = True):
    model.eval()
    correct = 0
    total = 0
    test_loss = 0
    with torch.no_grad():
        for batch_idx, (data, target) in enumerate(test_loader):
            if batch_idx * len(data) > test_size:
                break
            output = model(data)

            pred = output.data.max(1, keepdim=True)[1][0]
            
            correct += pred.eq(target.data.view_as(pred)).sum()
            total+=target.size(0)

            if batch_idx % 1000 == 0 and log:
              with open('log_baseline_test.csv', 'a') as f:
                writer = csv.writer(f)
                writer.writerow([batch_idx + test_size * 100, test_loss/(batch_idx+1), correct.item()/total])
              print("Epoch", epoch, 'iteration',batch_idx, 'Loss: %.3f | Acc: %.3f%% (%d/%d)'
                          % (test_loss/(batch_idx+1), 100.*correct/total, correct, total))

    return torch.div(correct, float(total))

In [ ]:
testloader = torch.utils.data.DataLoader(
    datasets.MNIST('../datasets/', train=True, download=True, 
                               transform=MNISTtransform), 
                               batch_size=1, shuffle=True)
'''Train CIFAR10 with PyTorch.'''
# import torch
# import torch.nn as nn
# import torch.optim as optim
# import torch.nn.functional as F
# import torch.backends.cudnn as cudnn

# import torchvision
# import torchvision.transforms as transforms

# import os
# import argparse
# import csv

# print('==> Preparing data..')
# transform_train = transforms.Compose([
#     transforms.RandomCrop(32, padding=4),
#     transforms.RandomHorizontalFlip(),
#     transforms.ToTensor(),
#     transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
# ])

# transform_test = transforms.Compose([
#     transforms.ToTensor(),
#     transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
# ])

# trainset = torchvision.datasets.CIFAR10(
#     root='./data', train=True, download=True, transform=transform_train)
# trainloader = torch.utils.data.DataLoader(
#     trainset, batch_size=128, shuffle=True, num_workers=2)

# testset = torchvision.datasets.CIFAR10(
#     root='./data', train=False, download=True, transform=transform_test)
# testloader = torch.utils.data.DataLoader(
#     testset, batch_size=1, shuffle=False, num_workers=2)

# classes = ('plane', 'car', 'bird', 'cat', 'deer',
#            'dog', 'frog', 'horse', 'ship', 'truck')

network_tester(model, testloader, 9000, 1)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:113: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


Epoch 1 iteration 0 Loss: 0.000 | Acc: 100.000% (1/1)
Epoch 1 iteration 1000 Loss: 0.000 | Acc: 99.500% (996/1001)
Epoch 1 iteration 2000 Loss: 0.000 | Acc: 99.600% (1993/2001)
Epoch 1 iteration 3000 Loss: 0.000 | Acc: 99.567% (2988/3001)
Epoch 1 iteration 4000 Loss: 0.000 | Acc: 99.625% (3986/4001)
Epoch 1 iteration 5000 Loss: 0.000 | Acc: 99.600% (4981/5001)
Epoch 1 iteration 6000 Loss: 0.000 | Acc: 99.517% (5972/6001)
Epoch 1 iteration 7000 Loss: 0.000 | Acc: 99.486% (6965/7001)
Epoch 1 iteration 8000 Loss: 0.000 | Acc: 99.525% (7963/8001)
Epoch 1 iteration 9000 Loss: 0.000 | Acc: 99.522% (8958/9001)


tensor(0.9952)

## ResNet-18 and MNIST

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.optim.lr_scheduler import StepLR

MNISTtransform = transforms.Compose([
                               transforms.ToTensor(),
                               transforms.Normalize((0.1307,), (0.3081,))])

testloader = torch.utils.data.DataLoader(
    datasets.MNIST('../datasets/', train=False, download=True, 
                               transform=MNISTtransform), 
                               batch_size=32, shuffle=True)

trainloader = torch.utils.data.DataLoader(
    datasets.MNIST('../datasets/', train=True, download=True, 
                               transform=MNISTtransform), 
                               batch_size=32, shuffle=True)

  0%|          | 0/9912422 [00:00<?, ?it/s]

Extracting ../datasets/MNIST/raw/train-images-idx3-ubyte.gz to ../datasets/MNIST/raw



  0%|          | 0/28881 [00:00<?, ?it/s]

Extracting ../datasets/MNIST/raw/train-labels-idx1-ubyte.gz to ../datasets/MNIST/raw



  0%|          | 0/1648877 [00:00<?, ?it/s]

Extracting ../datasets/MNIST/raw/t10k-images-idx3-ubyte.gz to ../datasets/MNIST/raw



  0%|          | 0/4542 [00:00<?, ?it/s]

Extracting ../datasets/MNIST/raw/t10k-labels-idx1-ubyte.gz to ../datasets/MNIST/raw



In [ ]:
'''ResNet in PyTorch.
For Pre-activation ResNet, see 'preact_resnet.py'.
Reference:
[1] Kaiming He, Xiangyu Zhang, Shaoqing Ren, Jian Sun
    Deep Residual Learning for Image Recognition. arXiv:1512.03385
'''
import torch
import torch.nn as nn
import torch.nn.functional as F


class BasicBlock(nn.Module):
    expansion = 1

    def __init__(self, in_planes, planes, stride=1):
        super(BasicBlock, self).__init__()
        self.conv1 = nn.Conv2d(
            in_planes, planes, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(planes)
        self.conv2 = nn.Conv2d(planes, planes, kernel_size=3,
                               stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(planes)

        self.shortcut = nn.Sequential()
        if stride != 1 or in_planes != self.expansion*planes:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_planes, self.expansion*planes,
                          kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(self.expansion*planes)
            )

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.bn2(self.conv2(out))
        out += self.shortcut(x)
        out = F.relu(out)
        return out


class Bottleneck(nn.Module):
    expansion = 4

    def __init__(self, in_planes, planes, stride=1):
        super(Bottleneck, self).__init__()
        self.conv1 = nn.Conv2d(in_planes, planes, kernel_size=1, bias=False)
        self.bn1 = nn.BatchNorm2d(planes)
        self.conv2 = nn.Conv2d(planes, planes, kernel_size=3,
                               stride=stride, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(planes)
        self.conv3 = nn.Conv2d(planes, self.expansion *
                               planes, kernel_size=1, bias=False)
        self.bn3 = nn.BatchNorm2d(self.expansion*planes)

        self.shortcut = nn.Sequential()
        if stride != 1 or in_planes != self.expansion*planes:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_planes, self.expansion*planes,
                          kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(self.expansion*planes)
            )

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = F.relu(self.bn2(self.conv2(out)))
        out = self.bn3(self.conv3(out))
        out += self.shortcut(x)
        out = F.relu(out)
        return out


class ResNet(nn.Module):
    def __init__(self, block, num_blocks, num_classes=10):
        super(ResNet, self).__init__()
        self.in_planes = 64

        self.conv1 = nn.Conv2d(1, 64, kernel_size=3,
                               stride=1, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(64)
        self.layer1 = self._make_layer(block, 64, num_blocks[0], stride=1)
        self.layer2 = self._make_layer(block, 128, num_blocks[1], stride=2)
        self.layer3 = self._make_layer(block, 256, num_blocks[2], stride=2)
        self.layer4 = self._make_layer(block, 512, num_blocks[3], stride=2)
        self.linear = nn.Linear(512*block.expansion, num_classes)

    def _make_layer(self, block, planes, num_blocks, stride):
        strides = [stride] + [1]*(num_blocks-1)
        layers = []
        for stride in strides:
            layers.append(block(self.in_planes, planes, stride))
            self.in_planes = planes * block.expansion
        return nn.Sequential(*layers)

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.layer4(out)
        out = F.avg_pool2d(out, 4)
        out = out.view(out.size(0), -1)
        out = self.linear(out)
        return out


def ResNet18():
    return ResNet(BasicBlock, [2, 2, 2, 2])


def ResNet34():
    return ResNet(BasicBlock, [3, 4, 6, 3])


def ResNet50():
    return ResNet(Bottleneck, [3, 4, 6, 3])


def ResNet101():
    return ResNet(Bottleneck, [3, 4, 23, 3])


def ResNet152():
    return ResNet(Bottleneck, [3, 8, 36, 3])

In [ ]:
'''Train CIFAR10 with PyTorch.'''
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torch.backends.cudnn as cudnn

import torchvision
import torchvision.transforms as transforms

import os
import argparse
import csv

device = 'cuda' if torch.cuda.is_available() else 'cpu'
best_acc = 0  # best test accuracy
start_epoch = 0  # start from epoch 0 or last checkpoint epoch

# Model
print('==> Building model..')
# net = VGG('VGG19')
net = ResNet18()

net = net.to(device)
if device == 'cuda':
    net = torch.nn.DataParallel(net)
    cudnn.benchmark = True

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001,
                      momentum=0.9, weight_decay=5e-4)
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=200)


# Training
def train(epoch):
    print('\nEpoch: %d' % epoch)
    net.train()
    train_loss = 0
    correct = 0
    total = 0
    for batch_idx, (inputs, targets) in enumerate(trainloader):
        inputs, targets = inputs.to(device), targets.to(device)
        optimizer.zero_grad()
        outputs = net(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()

        train_loss += loss.item()
        _, predicted = outputs.max(1)
        total += targets.size(0)
        correct += predicted.eq(targets).sum().item()
        if batch_idx % 50 == 0:
          with open('log_baseline_train.csv', 'a') as f:
                writer = csv.writer(f)
                writer.writerow([batch_idx + epoch * 400, train_loss/(batch_idx+1), correct/total])
          print("Epoch", epoch, 'iteration',batch_idx, 'Loss: %.3f | Acc: %.3f%% (%d/%d)'
                        % (train_loss/(batch_idx+1), 100.*correct/total, correct, total))


def test(epoch):
    global best_acc
    net.eval()
    test_loss = 0
    correct = 0
    total = 0
    with torch.no_grad():
        for batch_idx, (inputs, targets) in enumerate(testloader):
            inputs, targets = inputs.to(device), targets.to(device)
            outputs = net(inputs)
            loss = criterion(outputs, targets)

            test_loss += loss.item()
            _, predicted = outputs.max(1)
            total += targets.size(0)
            correct += predicted.eq(targets).sum().item()

            if batch_idx % 100 == 0:
              with open('log_baseline_test.csv', 'a') as f:
                writer = csv.writer(f)
                writer.writerow([batch_idx + epoch * 100, test_loss/(batch_idx+1), correct/total])
              print("Epoch", epoch, 'iteration',batch_idx, 'Loss: %.3f | Acc: %.3f%% (%d/%d)'
                          % (test_loss/(batch_idx+1), 100.*correct/total, correct, total))

    # Save checkpoint.
    acc = 100.*correct/total
    if acc > best_acc:
        print('Saving..')
        state = {
            'net': net.state_dict(),
            'acc': acc,
            'epoch': epoch,
        }
        if not os.path.isdir('checkpoint'):
            os.mkdir('checkpoint')
        torch.save(state, './checkpoint/ckpt.pth')
        best_acc = acc

with open('log_baseline_train.csv', 'w') as f:
        writer = csv.writer(f)
        writer.writerow(["iteration", "train_loss", "train_acc"])

with open('log_baseline_test.csv', 'w') as f:
        writer = csv.writer(f)
        writer.writerow(["iteration", "test_loss", "test_acc"])

for epoch in range(start_epoch, start_epoch+20):
    train(epoch)
    test(epoch)
    scheduler.step()

==> Building model..

Epoch: 0
Epoch 0 iteration 0 Loss: 2.453 | Acc: 9.375% (3/32)
Epoch 0 iteration 50 Loss: 1.778 | Acc: 43.934% (717/1632)
Epoch 0 iteration 100 Loss: 1.239 | Acc: 63.026% (2037/3232)
Epoch 0 iteration 150 Loss: 0.935 | Acc: 72.661% (3511/4832)
Epoch 0 iteration 200 Loss: 0.752 | Acc: 78.125% (5025/6432)
Epoch 0 iteration 250 Loss: 0.635 | Acc: 81.686% (6561/8032)
Epoch 0 iteration 300 Loss: 0.551 | Acc: 84.178% (8108/9632)
Epoch 0 iteration 350 Loss: 0.489 | Acc: 85.986% (9658/11232)
Epoch 0 iteration 400 Loss: 0.439 | Acc: 87.430% (11219/12832)
Epoch 0 iteration 450 Loss: 0.402 | Acc: 88.463% (12767/14432)
Epoch 0 iteration 500 Loss: 0.373 | Acc: 89.259% (14310/16032)
Epoch 0 iteration 550 Loss: 0.348 | Acc: 90.012% (15871/17632)
Epoch 0 iteration 600 Loss: 0.326 | Acc: 90.661% (17436/19232)
Epoch 0 iteration 650 Loss: 0.306 | Acc: 91.201% (18999/20832)
Epoch 0 iteration 700 Loss: 0.289 | Acc: 91.735% (20578/22432)
Epoch 0 iteration 750 Loss: 0.275 | Acc: 92.148% 

KeyboardInterrupt: ignored

In [ ]:
torch.save(net.state_dict(), "model_mnist.pth")

## RNN and Sentimental Analysis

In [ ]:
import csv

# file location (make sure to use your file location)
file_dir = '/content/'

# load csv file
def get_data():
    return csv.reader(open(file_dir + "training.1600000.processed.noemoticon.csv","rt", encoding="latin-1"))

# print only the first tweet
for i, line in enumerate(get_data()):
    if line[0] != '0':
        print(line[0], line[-1])
        break

4 I LOVE @Health4UandPets u guys r the best!! 


In [ ]:
import torchtext
glove = torchtext.vocab.GloVe(name="6B", dim=50)

def split_tweet(tweet):
    # separate punctuations
    tweet = tweet.replace(".", " . ") \
                 .replace(",", " , ") \
                 .replace(";", " ; ") \
                 .replace("?", " ? ")
    return tweet.lower().split()

glove = torchtext.vocab.GloVe(name="6B", dim=50, max_vectors=10000)

.vector_cache/glove.6B.zip: 862MB [02:42, 5.31MB/s]                           
100%|█████████▉| 9999/10000 [00:00<00:00, 40258.50it/s]


In [ ]:
import torch
import torch.nn as nn

def get_tweet_vectors(glove_vector):
    train, valid, test = [], [], []
    for i, line in enumerate(get_data()):
        tweet = line[-1]
        if i % 59 == 0:
            # obtain an embedding for the entire tweet
            tweet_emb = sum(glove_vector[w] for w in split_tweet(tweet))
            # generate a label: 1 = happy, 0 = sad
            label = torch.tensor(int(line[0] == "4")).long()
            # place the data set in either the training, validation, or test set
            if i % 5 < 3:
                train.append((tweet_emb, label)) # 60% training
            elif i % 5 == 4:
                valid.append((tweet_emb, label)) # 20% validation
            else:
                test.append((tweet_emb, label)) # 20% test
    return train, valid, test

In [ ]:
import torchtext

glove = torchtext.vocab.GloVe(name="6B", dim=50)

train, valid, test = get_tweet_vectors(glove)

train_loader = torch.utils.data.DataLoader(train, batch_size=128, shuffle=True)
valid_loader = torch.utils.data.DataLoader(valid, batch_size=128, shuffle=True)
test_loader = torch.utils.data.DataLoader(test, batch_size=128, shuffle=True)

In [ ]:
def get_tweet_words(glove_vector):
    train, valid, test = [], [], []
    for i, line in enumerate(get_data()):
        if i % 29 == 0:
            tweet = line[-1]
            idxs = [glove_vector.stoi[w]        # lookup the index of word
                    for w in split_tweet(tweet)
                    if w in glove_vector.stoi] # keep words that has an embedding
            if not idxs: # ignore tweets without any word with an embedding
                continue
            idxs = torch.tensor(idxs) # convert list to pytorch tensor
            label = torch.tensor(int(line[0] == "4")).long()
            if i % 5 < 3:
                train.append((idxs, label))
            elif i % 5 == 4:
                valid.append((idxs, label))
            else:
                test.append((idxs, label))
    return train, valid, test

train, valid, test = get_tweet_words(glove)

In [ ]:
import matplotlib.pyplot as plt

def train_rnn_network(model, train, valid, num_epochs=5, learning_rate=1e-5):
    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate, momentum = 0.9)
    losses, train_acc, valid_acc = [], [], []
    epochs = []
    for epoch in range(num_epochs):
        i = 0
        for tweets, labels in train:
            i += 1
            optimizer.zero_grad()
            pred = model(tweets)
            loss = criterion(pred, labels)
            loss.backward()
            optimizer.step()
            if i % 100 == 1:
                train_acc.append(get_accuracy(model, train_loader))
                valid_acc.append(get_accuracy(model, valid_loader))
                print("Epoch %d; Iteration %d; Loss %f; Train Acc %f; Val Acc %f" % (
                      epoch+1, i, loss, train_acc[-1], valid_acc[-1]))
                with open('log_baseline_test2.csv', 'a') as f:
                    writer = csv.writer(f)
                    writer.writerow([epoch * 100 + i, loss, valid_acc[-1]])
                  
        losses.append(float(loss))

        epochs.append(epoch)
        train_acc.append(get_accuracy(model, train_loader))
        valid_acc.append(get_accuracy(model, valid_loader))
        print("End of Epoch %d; Loss %f; Train Acc %f; Val Acc %f" % (
              epoch+1, loss, train_acc[-1], valid_acc[-1]))

def get_accuracy(model, data_loader):
    correct, total = 0, 0
    for tweets, labels in data_loader:
        output = model(tweets)
        pred = output.max(1, keepdim=True)[1]
        correct += pred.eq(labels.view_as(pred)).sum().item()
        total += labels.shape[0]
    return correct / total

In [ ]:
class TweetRNN(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(TweetRNN, self).__init__()
        self.emb = nn.Embedding.from_pretrained(glove.vectors)
        self.hidden_size = hidden_size
        self.rnn = nn.RNN(input_size, hidden_size, batch_first=True)
        self.fc = nn.Linear(hidden_size, num_classes)
    
    def forward(self, x):
        # Look up the embedding
        x = self.emb(x)
        # Set an initial hidden state
        h0 = torch.zeros(1, x.size(0), self.hidden_size)
        # Forward propagate the RNN
        out, _ = self.rnn(x, h0)
        # Pass the output of the last time step to the classifier
        out = self.fc(out[:, -1, :])
        return out

model = TweetRNN(50, 50, 2)

In [ ]:
from torch.nn.utils.rnn import pad_sequence

tweet_padded = pad_sequence([tweet for tweet, label in train[:10]],
                            batch_first=True)

In [ ]:
import random

class TweetBatcher:
    def __init__(self, tweets, batch_size=32, drop_last=False):
        # store tweets by length
        self.tweets_by_length = {}
        for words, label in tweets:
            # compute the length of the tweet
            wlen = words.shape[0]
            # put the tweet in the correct key inside self.tweet_by_length
            if wlen not in self.tweets_by_length:
                self.tweets_by_length[wlen] = []
            self.tweets_by_length[wlen].append((words, label),)
         
        #  create a DataLoader for each set of tweets of the same length
        self.loaders = {wlen : torch.utils.data.DataLoader(
                                    tweets,
                                    batch_size=batch_size,
                                    shuffle=True,
                                    drop_last=drop_last) # omit last batch if smaller than batch_size
            for wlen, tweets in self.tweets_by_length.items()}
        
    def __iter__(self): # called by Python to create an iterator
        # make an iterator for every tweet length
        iters = [iter(loader) for loader in self.loaders.values()]
        while iters:
            # pick an iterator (a length)
            im = random.choice(iters)
            try:
                yield next(im)
            except StopIteration:
                # no more elements in the iterator, remove it
                iters.remove(im)

In [ ]:
def get_accuracy(model, data_loader):
    correct, total = 0, 0
    for tweets, labels in data_loader:
        output = model(tweets)
        pred = output.max(1, keepdim=True)[1]
        correct += pred.eq(labels.view_as(pred)).sum().item()
        total += labels.shape[0]
    return correct / total

test_loader = TweetBatcher(test, batch_size=64, drop_last=False)
get_accuracy(model, test_loader)

0.4944403937294933

In [ ]:
net = TweetRNN(50, 100, 2)
train_loader = TweetBatcher(train, batch_size=64, drop_last=True)
valid_loader = TweetBatcher(valid, batch_size=64, drop_last=False)
train_rnn_network(net, train_loader, valid_loader, num_epochs=40, learning_rate=5e-3)
get_accuracy(net, test_loader)

Epoch 1; Iteration 1; Loss 0.702942; Train Acc 0.508663; Val Acc 0.509484
Epoch 1; Iteration 101; Loss 0.688663; Train Acc 0.590064; Val Acc 0.594930
Epoch 1; Iteration 201; Loss 0.644955; Train Acc 0.621755; Val Acc 0.620190
Epoch 1; Iteration 301; Loss 0.677277; Train Acc 0.635364; Val Acc 0.640890
Epoch 1; Iteration 401; Loss 0.643282; Train Acc 0.621661; Val Acc 0.625935
End of Epoch 1; Loss 0.619829; Train Acc 0.664409; Val Acc 0.669433
Epoch 2; Iteration 1; Loss 0.691065; Train Acc 0.661983; Val Acc 0.658672
Epoch 2; Iteration 101; Loss 0.629911; Train Acc 0.669512; Val Acc 0.672898
Epoch 2; Iteration 201; Loss 0.626382; Train Acc 0.560736; Val Acc 0.564290
Epoch 2; Iteration 301; Loss 0.670268; Train Acc 0.646705; Val Acc 0.651924
Epoch 2; Iteration 401; Loss 0.583889; Train Acc 0.673513; Val Acc 0.673263
End of Epoch 2; Loss 0.620507; Train Acc 0.666677; Val Acc 0.665603
Epoch 3; Iteration 1; Loss 0.593867; Train Acc 0.658424; Val Acc 0.656666
Epoch 3; Iteration 101; Loss 0.558

0.7002369668246445

In [ ]:
torch.save(net.state_dict(), "model_sentimental.pth")

In [ ]:
class TweetRNNM(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(TweetRNNM, self).__init__()
        self.hidden_size = hidden_size
        self.emb = nn.Embedding.from_pretrained(glove.vectors)
        self.hidden_size = hidden_size
        self.rnn = nn.RNN(input_size, hidden_size, batch_first=True)
        #self.fc = nn.Linear(hidden_size, num_classes)

        self.uvect = [2*(torch.rand(hidden_size,1) - 0.5) for i in range(0,num_classes)] # 
        crb1 = crossbar(device_params)
        # Can test using more than 1 crossbar linear layers.
        # Easiest implementation is to create a crossbar for each linear layer
        self.fc1 = Linear(hidden_size, num_classes,crb1,self.uvect)
        self.fc1.use_cb(True)
        #self.fc2 = nn.Linear(64*2*2, 10)
        self.traincount = 0
    
    def forward(self, x):
        # Look up the embedding
        x = self.emb(x)
        # Set an initial hidden state
        h0 = torch.zeros(1, x.size(0), self.hidden_size)
        # Forward propagate the RNN
        out, _ = self.rnn(x, h0)
        # Pass the output of the last time step to the classifier
        out = out[:, -1, :]
        out = self.fc1(out.view(self.hidden_size, 1)).t()
        out = F.log_softmax(out)
        return out

In [ ]:
state_dict = torch.load('model_sentimental.pth')
temp_model = TweetRNN(50, 100, 2)
temp_model.load_state_dict(torch.load('model_sentimental.pth'))

# from collections import OrderedDict
# new_state_dict = OrderedDict()
# for k, v in state_dict.items():
#     name = k[7:] # remove `module.`
#     new_state_dict[name] = v
# # load params
# temp_model.load_state_dict(new_state_dict)

<All keys matched successfully>

In [ ]:
state_dict = torch.load('model_sentimental.pth')
model = TweetRNNM(50, 100, 2)
model.load_state_dict(torch.load('model_sentimental.pth'))

RuntimeError: ignored

In [ ]:
# crossbar update
device_params = {"Vdd": 1.8,
                 "r_wl": 20,
                 "r_bl": 20,
                 "m": 200,
                 "n": 200,
                 "r_on_mean": 1e4,
                 "r_on_stddev": 1e3,
                 "r_off_mean": 1e5,
                 "r_off_stddev": 1e4,
                 "dac_resolution": 5,
                 "adc_resolution": 8.3,
                 "device_resolution": 6,
                 "bias_scheme": 1/3,
                 "tile_rows": 4,
                 "tile_cols": 4,
                 "r_cmos_line": 600,
                 "r_cmos_transistor": 20, 
                 "p_stuck_on": 0.01,
                 "p_stuck_off": 0.01}
crb_new = crossbar(device_params)

model.fc1.W = torch.nn.parameter.Parameter(temp_model.fc.weight.data)
model.fc1.b = torch.nn.parameter.Parameter(temp_model.fc.bias.data)

model.fc1.cb = crb_new
model.fc1.remap()

In [ ]:
model.eval()

TweetRNNM(
  (emb): Embedding(400000, 50)
  (rnn): RNN(50, 100, batch_first=True)
  (fc1): Linear()
)

In [ ]:
def network_tester(model, test_loader, test_size, epoch, log = True):
    model.eval()
    correct = 0
    total = 0
    test_loss = 0
    with torch.no_grad():
        for batch_idx, (data, target) in enumerate(test_loader):
            if batch_idx * len(data) > test_size:
                break
            output = model(data)

            pred = output.data.max(1, keepdim=True)[1][0]
            
            correct += pred.eq(target.data.view_as(pred)).sum()
            total+=target.size(0)

            if batch_idx % 1000 == 0 and log:
              with open('log_baseline_test.csv', 'a') as f:
                writer = csv.writer(f)
                writer.writerow([batch_idx + test_size * 100, test_loss/(batch_idx+1), correct.item()/total])
              print("Epoch", epoch, 'iteration',batch_idx, 'Loss: %.3f | Acc: %.3f%% (%d/%d)'
                          % (test_loss/(batch_idx+1), 100.*correct/total, correct, total))

    return torch.div(correct, float(total))

In [ ]:
testloader = torch.utils.data.DataLoader(test, batch_size=1, shuffle=True)
network_tester(model, testloader, 9000, 1)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:29: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


Epoch 1 iteration 0 Loss: 0.000 | Acc: 100.000% (1/1)
Epoch 1 iteration 1000 Loss: 0.000 | Acc: 68.731% (688/1001)
Epoch 1 iteration 2000 Loss: 0.000 | Acc: 68.466% (1370/2001)
Epoch 1 iteration 3000 Loss: 0.000 | Acc: 69.110% (2074/3001)
Epoch 1 iteration 4000 Loss: 0.000 | Acc: 69.458% (2779/4001)
Epoch 1 iteration 5000 Loss: 0.000 | Acc: 69.086% (3455/5001)
Epoch 1 iteration 6000 Loss: 0.000 | Acc: 69.522% (4172/6001)
Epoch 1 iteration 7000 Loss: 0.000 | Acc: 69.761% (4884/7001)
Epoch 1 iteration 8000 Loss: 0.000 | Acc: 69.654% (5573/8001)
Epoch 1 iteration 9000 Loss: 0.000 | Acc: 69.737% (6277/9001)


tensor(0.6974)